In [6]:
import pandas as pd
import os
import sys
import glob
import numpy as np

sys.path.insert(0, '../')

In [7]:
np.sqrt(.3)

0.5477225575051661

### Run through stocks we pulled data from IEX for. Update the .csv file with this historical data

In [44]:
def add_historic_data_to_current_csv():
    current_data_file_path = '../data/historical_etf_data'
#    current_data_file_path = '../data/historical_stock_data'
    augmented_data_file_path = '../data/updated_historical_stock_and_etf_data'
    historic_data_file_path = '../data/price-volume-data-for-all-us-stocks-etfs/ETFs'
#    historic_data_file_path = '../data/price-volume-data-for-all-us-stocks-etfs/Stocks'    

    for file in glob.glob(f'{current_data_file_path}/*.csv'):
        print(f'File = {file}')
        stock_name = file.rsplit("/")[-1].split('_')[0].lower() # note: store all stock names as uppercase
        found_historical_stock = False
        # find the stock name in historical kaggle dataa
        for historical_file in glob.glob(f'{historic_data_file_path}/*.txt'):
            historical_stock_name = historical_file.split("/")[-1].split(".")[0]
            
            if stock_name == historical_stock_name:
                print('found historical stock', historical_stock_name)
                # read in current data
                current_stock_data = pd.read_csv(f"{file}")
                print(historical_file, 'historical_file')
                historical_stock_data = pd.read_csv(f"{historical_file}")
                historical_stock_data.columns = map(str.lower, historical_stock_data.columns)

                oldest_date_of_new_data = current_stock_data.date.min()

                # get the new data we don't have yet
                historical_stock_data = historical_stock_data[historical_stock_data.date < oldest_date_of_new_data]
                historical_stock_data.drop('openint', inplace=True, axis=1)


                # concat historical with current data
                combined_stock_df = pd.concat([current_stock_data, historical_stock_data])
                most_recent_date = combined_stock_df.date.max()
                most_recent_year, most_recent_month, most_recent_day = most_recent_date.split('-')

                oldest_date = combined_stock_df.date.min()
                oldest_year, oldest_month, oldest_day = oldest_date.split('-')
                
               # delete the old file only if we haven't  updated it
                delete_old_csv(stock_name, augmented_data_file_path)
                
                # save the file
                combined_stock_df.to_csv(f'{augmented_data_file_path}/{stock_name.lower()}_{int(oldest_year)}-{int(oldest_month)}-{int(oldest_day)}_to_{int(most_recent_year)}-{int(most_recent_month)}-{int(most_recent_day)}.csv',
                                         index=False)
                found_historical_stock = True
             
                break

        # # if we don't have historical data , save the original data into the augmented folder
        if not found_historical_stock:
            print(file.split("/")[-1])

            current_stock_data = pd.read_csv(f"{file}")
            stock_name = file.split("/")[-1]
            
            current_stock_data.to_csv(f"{augmented_data_file_path}/{stock_name}",
                                             index=False)
        # if os.path.isfile(f'{augmented_data_file_path}/{stock_name.lower()}_{int(oldest_year)}-{int(oldest_month)}-{int(oldest_day)}_to_{int(most_recent_year)}-{int(most_recent_month)}-{int(most_recent_day)}.csv'):
        #     pass
        # else:
        #     current_stock_data = pd.read_csv(f"{file}")
        #     current_stock_data.to_csv(f"{augmented_data_file_path}/)



In [45]:
def delete_old_csv(stock_name, folder_path):
    for file in glob.glob(f'{folder_path}/*.csv'):
        if stock_name in file:
            os.remove(file)

In [46]:
add_historic_data_to_current_csv()

File = ../data/historical_etf_data/ibuy_2017-1-1_to_2020-6-26.csv
ibuy_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/dia_2017-1-1_to_2020-6-26.csv
dia_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/bjk_2017-1-1_to_2020-6-26.csv
bjk_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/hpt_2017-1-1_to_2020-4-29.csv
hpt_2017-1-1_to_2020-4-29.csv
File = ../data/historical_etf_data/pbj_2017-1-1_to_2020-6-26.csv
pbj_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/spy_2017-1-1_to_2020-6-26.csv
spy_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/sh_2017-01-03_to_2020-6-26.csv
sh_2017-01-03_to_2020-6-26.csv
File = ../data/historical_etf_data/uso_2017-1-1_to_2020-6-26.csv
uso_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/vti_2017-1-1_to_2020-6-26.csv
vti_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/pej_2017-1-1_to_2020-6-26.csv
pej_2017-1-1_to_2020-6-26.csv
File = ../data/historical_etf_data/vfh_2017-1-